In [143]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import pandas as pd
from sklearn.model_selection import train_test_split
from torchtext.vocab import GloVe
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [144]:
# %%¯¸
df_train = pd.read_csv("../../data/Text-To-SQL-COL775/train.csv")
df_tables = pd.read_json("../../data/Text-To-SQL-COL775/tables.json")


In [104]:
df_tables

,column_names,column_names_original,column_types,db_id,foreign_keys,primary_keys,table_names,table_names_original
0,"[[-1, *], [0, perpetrator id], [0, people id],...","[[-1, *], [0, Perpetrator_ID], [0, People_ID],...","[text, number, number, text, number, text, tex...",perpetrator,"[[2, 9]]","[1, 9]","[perpetrator, people]","[perpetrator, people]"
1,"[[-1, *], [0, building], [0, room number], [0,...","[[-1, *], [0, building], [0, room_number], [0,...","[text, text, text, number, text, text, number,...",college_2,"[[9, 4], [13, 4], [19, 1], [20, 2], [15, 7], [...","[1, 4, 7, 11, 15, 22, 27, 31, 37, 39, 45]","[classroom, department, course, instructor, se...","[classroom, department, course, instructor, se..."
2,"[[-1, *], [0, id], [0, city], [0, country], [0...","[[-1, *], [0, id], [0, City], [0, Country], [0...","[text, number, text, text, text, text, text, n...",flight_company,"[[20, 7], [19, 1]]","[1, 7, 13]","[airport, operate company, flight]","[airport, operate_company, flight]"
3,"[[-1, *], [0, institution id], [0, name], [0, ...","[[-1, *], [0, instID], [0, name], [0, country]...","[text, number, text, text, number, text, text,...",icfp_1,"[[11, 7], [10, 1], [9, 4]]","[1, 4, 7, 9]","[institution, authors, papers, authorship count]","[Inst, Authors, Papers, Authorship]"
4,"[[-1, *], [0, body builder id], [0, people id]...","[[-1, *], [0, Body_Builder_ID], [0, People_ID]...","[text, number, number, number, number, number,...",body_builder,"[[2, 6]]","[1, 6]","[body builder, people]","[body_builder, people]"
...,...,...,...,...,...,...,...,...
161,"[[-1, *], [0, employee ssn], [0, project numbe...","[[-1, *], [0, Essn], [0, Pno], [0, Hours], [1,...","[text, number, number, number, text, text, tex...",company_1,[],"[1, 7, 15, 19, 22, 27]","[works on, employee, department, project, depe...","[works_on, employee, department, project, depe..."
162,"[[-1, *], [0, workshop id], [0, date], [0, ven...","[[-1, *], [0, Workshop_ID], [0, Date], [0, Ven...","[text, number, text, text, text, number, numbe...",workshop_paper,"[[10, 1], [9, 5]]","[1, 5, 9]","[workshop, submission, acceptance]","[workshop, submission, Acceptance]"
163,"[[-1, *], [0, item id], [0, title], [1, a id],...","[[-1, *], [0, i_id], [0, title], [1, a_id], [1...","[text, number, text, number, number, number, n...",epinions_1,"[[5, 1], [4, 8], [11, 8], [10, 8]]","[1, 3, 8]","[item, review, useracct, trust]","[item, review, useracct, trust]"
164,"[[-1, *], [0, party id], [0, party theme], [0,...","[[-1, *], [0, Party_ID], [0, Party_Theme], [0,...","[text, number, text, text, text, text, number,...",party_host,"[[11, 1], [12, 7]]","[1, 7, 11]","[party, host, party host]","[party, host, party_host]"


In [105]:
df_train

,db_id,query,question
0,geo,SELECT MAX ( highest_elevation ) FROM highlow,what is the height of the highest point in the...
1,geo,SELECT COUNT ( city_name ) FROM city,how many cities are there in usa
2,cre_Doc_Tracking_DB,SELECT DISTINCT location_code FROM Document_lo...,What are the different location codes for docu...
3,program_share,SELECT name FROM program ORDER BY launch DESC ...,find the name of the program that was launched...
4,college_2,SELECT name FROM instructor ORDER BY salary DE...,Give the name of the highest paid instructor.
...,...,...,...
7749,tvshow,SELECT T1.series_name FROM TV_Channel AS T1 JO...,What is the TV Channel of TV series with Episo...
7750,restaurants,"SELECT t2.house_number , t1.name FROM LOCATI...",what is a good restaurant in the bay area ?
7751,music_2,SELECT TYPE FROM vocals GROUP BY TYPE ORDER BY...,Which vocal type is the most frequently appear...
7752,scholar,SELECT DISTINCT t3.venueid FROM writes AS t2 J...,Where did li dong publish in 2016 ?


In [106]:
df_test = pd.read_csv("../../data/sql_query.csv")

In [107]:

# %%
train_file = "../../data/Text-To-SQL-COL775/train.csv"
table_file = "../../data/Text-To-SQL-COL775/tables.json"
val_file = "../../data/Text-To-SQL-COL775/val.csv"
test_file = "../../data/sql_query.csv"


In [108]:
# question = df_train.iloc[0]['question'] # question extracted
# query = df_train.iloc[0]['query'] # query extracted
# schema = df_train.iloc[0]['db_id'] # get database id
# # Get the table and column names for the database schema
# tables = df_tables[df_tables['db_id'] == schema]['table_names_original'].tolist() # entire table data for that db_id
# columns = df_tables[df_tables['db_id'] == schema]['column_names_original'].tolist()
# # Convert the table and column names to a list of strings
# tables = [item for sublist in tables for item in sublist]
# columns = [item for sublist in columns for item in sublist]
# tables = [str(table) for table in tables]
# columns = [str(column) for column in columns]
# # Concatenate the table and column names to the question and query inputs
# question += ' ' + ' '.join(tables) + ' ' + ' '.join(columns)
# query += ' ' + ' '.join(tables) + ' ' + ' '.join(columns)


In [117]:
class SpiderDataset(data.Dataset):
    def __init__(self, data_file, table_file):
        self.data = pd.read_csv(data_file)
        self.table_data = pd.read_json(table_file)
        self.vocab = GloVe(name='6B', dim=100, unk_init=torch.Tensor.normal_)
        
    def __len__(self):
        return len(self.data)

    
    def __getitem__(self, index):
        question = self.data.iloc[index]['question'] # question extracted
        query = self.data.iloc[index]['query'] # query extracted
        schema = self.data.iloc[index]['db_id'] # get database id
        # Get the table and column names for the database schema
        tables = self.table_data[self.table_data['db_id'] == schema]['table_names_original'].tolist() # entire table data for that db_id
        columns = self.table_data[self.table_data['db_id'] == schema]['column_names_original'].tolist()
        # Convert the table and column names to a list of strings
        tables = [item for sublist in tables for item in sublist]
        columns = [item for sublist in columns for item in sublist]
        tables = [str(table) for table in tables]
        columns = [str(column) for column in columns]
        # Concatenate the table and column names to the question and query inputs
        question += ' ' + ' '.join(tables) + ' ' + ' '.join(columns)
        query += ' ' + ' '.join(tables) + ' ' + ' '.join(columns)

        return question, query, schema
    
    def collate_fn(self, batch):
        questions, queries, schemas = zip(*batch)
        # Tokenize and convert to tensors
        question_seq = [torch.tensor([self.vocab.stoi.get(token, self.vocab.stoi['unk']) for token in question.split() if token != '']) for question in questions]
        query_seq = [torch.tensor([self.vocab.stoi.get(token, self.vocab.stoi['unk']) for token in query.split() if token != '']) for query in queries]
        schema_seq = [torch.tensor([self.vocab.stoi.get(token, self.vocab.stoi['unk']) for token in schema.split() if token != '']) for schema in schemas]
        # Pad sequences to the same length
        question_seq = nn.utils.rnn.pad_sequence(question_seq, batch_first=True, padding_value=0)
        query_seq = nn.utils.rnn.pad_sequence(query_seq, batch_first=True, padding_value=0)
        schema_seq = nn.utils.rnn.pad_sequence(schema_seq, batch_first=True, padding_value=0)

        return question_seq, query_seq, schema_seq


In [118]:
# Create instances of the SpiderDataset class for each set
train_dataset = SpiderDataset(train_file, table_file)
val_dataset = SpiderDataset(val_file, table_file)
test_dataset = SpiderDataset(test_file, table_file)

In [137]:
train_dataset[2]

("What are the different location codes for documents? Ref_Document_Types Ref_Calendar Ref_Locations Roles All_Documents Employees Document_Locations Documents_to_be_Destroyed [-1, '*'] [0, 'Document_Type_Code'] [0, 'Document_Type_Name'] [0, 'Document_Type_Description'] [1, 'Calendar_Date'] [1, 'Day_Number'] [2, 'Location_Code'] [2, 'Location_Name'] [2, 'Location_Description'] [3, 'Role_Code'] [3, 'Role_Name'] [3, 'Role_Description'] [4, 'Document_ID'] [4, 'Date_Stored'] [4, 'Document_Type_Code'] [4, 'Document_Name'] [4, 'Document_Description'] [4, 'Other_Details'] [5, 'Employee_ID'] [5, 'Role_Code'] [5, 'Employee_Name'] [5, 'Gender_MFU'] [5, 'Date_of_Birth'] [5, 'Other_Details'] [6, 'Document_ID'] [6, 'Location_Code'] [6, 'Date_in_Location_From'] [6, 'Date_in_Locaton_To'] [7, 'Document_ID'] [7, 'Destruction_Authorised_by_Employee_ID'] [7, 'Destroyed_by_Employee_ID'] [7, 'Planned_Destruction_Date'] [7, 'Actual_Destruction_Date'] [7, 'Other_Details']",
 "SELECT DISTINCT location_code FR

In [138]:
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=train_dataset.collate_fn)


In [145]:
for i, batch in enumerate(train_loader):
    
    qt, qy, sch = [tensor.to(device) for tensor in batch]
    break

In [146]:
qt.shape, qy.shape, sch.shape

(torch.Size([2, 84]), torch.Size([2, 92]), torch.Size([2, 1]))

In [148]:
model(qt, sch )


tensor([[[-0.0015,  0.0926,  0.0375,  ..., -0.0717,  0.0031, -0.0512],
         [-0.0216,  0.0792,  0.0453,  ..., -0.0565,  0.0082, -0.0385],
         [-0.0315,  0.0605,  0.0532,  ..., -0.0577,  0.0109, -0.0473],
         ...,
         [-0.0250,  0.0145,  0.0701,  ..., -0.0471,  0.0004, -0.0715],
         [-0.0250,  0.0144,  0.0701,  ..., -0.0470,  0.0004, -0.0716],
         [-0.0250,  0.0143,  0.0701,  ..., -0.0470,  0.0004, -0.0716]],

        [[-0.0508,  0.0684,  0.0403,  ..., -0.0801,  0.0340, -0.0170],
         [-0.0372,  0.0750,  0.0418,  ..., -0.0809,  0.0292, -0.0283],
         [-0.0325,  0.0604,  0.0520,  ..., -0.0743,  0.0241, -0.0466],
         ...,
         [-0.0069,  0.0973,  0.0413,  ..., -0.0432, -0.0205, -0.0612],
         [-0.0069,  0.0973,  0.0413,  ..., -0.0432, -0.0205, -0.0612],
         [-0.0069,  0.0973,  0.0413,  ..., -0.0432, -0.0205, -0.0612]]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [134]:
# class Seq2SeqLSTM(nn.Module):
#     def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):
#         super(Seq2SeqLSTM, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embedding_dim)
#         self.encoder = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True)
#         self.decoder = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True)
#         self.linear = nn.Linear(hidden_dim, vocab_size)
        
#     def forward(self, input_seq, target_seq, input_lengths):
#         embedded_input = self.embedding(input_seq)
#         packed_input = pack_padded_sequence(embedded_input, input_lengths, batch_first=True, enforce_sorted=False)
#         packed_output, hidden = self.encoder(packed_input)
#         encoder_output, _ = pad_packed_sequence(packed_output, batch_first=True)
#         decoder_output, _ = self.decoder(embedded_input, hidden)
#         output = self.linear(decoder_output)
#         return output

class Seq2SeqLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):
        super(Seq2SeqLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.decoder = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, vocab_size)
        
    def forward(self, input_seq, target_seq):
        embedded_input = self.embedding(input_seq)
        encoded_seq, (hidden, cell) = self.encoder(embedded_input)
        decoded_seq, _ = self.decoder(embedded_input, (hidden, cell))
        output_seq = self.linear(decoded_seq)
        return output_seq


# class Seq2SeqLSTM(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size, num_layers=1, dropout=0.2):
#         super(Seq2SeqLSTM, self).__init__()
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.dropout = dropout
#         self.encoder = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
#         self.decoder = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
#         self.fc = nn.Linear(hidden_size, output_size)
    
#     def forward(self, x):
#         # x is of shape (batch_size, seq_length, input_size)
#         # Set initial states

#         h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
#         c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
#         # Forward propagate encoder
#         encoder_output, (hidden, cell) = self.encoder(x, (h0, c0))
#         # Forward propagate decoder
#         decoder_output, _ = self.decoder(encoder_output, (hidden, cell))
#         # Decode the hidden state of the last time step
#         out = self.fc(decoder_output[:, -1, :])
#         return out

In [133]:
vocab_size

400000

In [136]:
# Define hyperparameters
vocab_size = len(train_dataset.vocab)   
input_size = 100
hidden_size = 256
num_layers = 1
num_epochs = 5
learning_rate = 0.001
batch_size = 32
#dropout = 0.2
output_size = vocab_size


#model = Seq2SeqLSTM(input_size, hidden_size, output_size, num_layers, dropout).to(device)
#model = Seq2SeqLSTM(input_size, hidden_size, output_size, num_layers).to(device)
model = Seq2SeqLSTM(vocab_size, embedding_dim=100, hidden_dim=256, num_layers=2).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Create data loaders for each set using the SpiderDataset.collate_fn method
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=train_dataset.collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=val_dataset.collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=test_dataset.collate_fn)

In [116]:

for epoch in range(num_epochs):
    train_loss = 0.0
    val_loss = 0.0
    model.train()
    for i, batch in enumerate(tqdm(train_loader)):
        optimizer.zero_grad()
        questions, queries, input_lengths = batch
        print(questions.shape, queries.shape, input_lengths.shape)
        output = model(questions, queries[:,:-1])
        target = queries[:,1:]
        loss = loss_fn(output.reshape(-1, vocab_size), target.reshape(-1))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    model.eval()
    with torch.no_grad():
        for i, batch in enumerate(val_loader):
            questions, queries, input_lengths = batch
            output = model(questions, queries[:,:-1])
            target = queries[:,1:]
            loss = loss_fn(output.reshape(-1, vocab_size), target.reshape(-1))
            val_loss += loss.item()
        val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}")


  0%|          | 0/122 [00:00<?, ?it/s]

torch.Size([64, 213]) torch.Size([64, 212]) torch.Size([64, 1])


TypeError: forward() takes 2 positional arguments but 3 were given

In [115]:
model(questions, queries[:,:-1], input_lengths)

TypeError: forward() takes 2 positional arguments but 4 were given

In [100]:
queries[:,:-1]

tensor([[201534, 201534,      1,  ...,      0,      0,      0],
        [201534,  13697, 201534,  ...,      0,      0,      0],
        [201534, 201534,      1,  ...,      0,      0,      0],
        ...,
        [201534, 201534,      1,  ...,      0,      0,      0],
        [201534, 201534, 201534,  ...,      0,      0,      0],
        [201534, 201534, 201534,  ...,      0,      0,      0]])

In [99]:
queries[:,1:]

tensor([[201534,      1, 201534,  ...,      0,      0,      0],
        [ 13697, 201534, 201534,  ...,      0,      0,      0],
        [201534,      1, 201534,  ...,      0,      0,      0],
        ...,
        [201534,      1, 201534,  ...,      0,      0,      0],
        [201534, 201534,  33537,  ...,      0,      0,      0],
        [201534, 201534, 201534,  ...,      0,      0,      0]])

In [77]:
questions.shape

torch.Size([64, 220])

In [63]:
output.reshape(-1, vocab_size).shape, target.reshape(-1).shape

(torch.Size([3440, 400000]), torch.Size([3392]))

In [53]:
from torchtext.vocab import GloVe

# Load the GloVe word embeddings
#vocab = GloVe(name='6B', dim=100)

# Check if the 'How' token is present in the vocabulary
if 'How' in vocab.stoi:
    print("'How' is present in the vocabulary")
else:
    print("'How' is not present in the vocabulary")

'How' is present in the vocabulary


In [52]:
from torchtext.vocab import GloVe

# Load the GloVe word embeddings
vocab = GloVe(name='6B', dim=100)

# Add the 'How' token to the vocabulary with a new embedding vector
new_token = 'How'
new_embedding = torch.randn(1, 100)  # Generate a new random embedding vector for the token
vocab.stoi[new_token] = len(vocab.stoi)  # Add the new token to the vocabulary
vocab.vectors = torch.cat((vocab.vectors, new_embedding), dim=0)  # Add the new embedding vector to the GloVe embeddings tensor

# Check if the 'How' token is now present in the vocabulary
if 'How' in vocab.stoi:
    print("'How' has been added to the vocabulary")
else:
    print("Error: 'How' was not added to the vocabulary")


'How' has been added to the vocabulary
